# **Basics of Mobile Robotics Project** 
## **EPFL - December 2023**
### **Professor: Francesco Mondada**
### **Authors:**
- Evangelista Santiago Roberto

- Giovine Angelo

- He Weifeng

- Syla Valmir

## **Tables of contents**

- [Demo Videos](#Demo-Videos)

- [Introduction](#Introduction)
  - [Project description and details](#Project-description-and-details)
  - [Modules and libraries required](#Modules-and-libraries-required)

- [Environment](#Environment)

- [Vision](#Vision)
  - [Camera](#Camera)
  - [Environment mapping](#Environment-mapping)
  - [Global obstacle detection](#Global-obstacle-detection)
  - [Robot detection](#Robot-detection)
  - [Goal detection](#Goal-detection)

  

- [Global Navigation](#Global-Navigation)
  - [Visibility graph](#Visibility-graph)



- [Local Navigation](#Local-Navigation)
  - [Local obstacle detection](#Local-obstacle-detection)
  - [Kidnaping](#Kidnaping)

- [Filtering](#Filtering:-Extended-Kalman-Filter)
  - [Why Kalman?](#Deriving-the-state-space-model-of-the-Thymio-Robot)
  - [Deriving the state space model of the Thymio Robot](#Deriving-the-state-space-model-of-the-Thymio-Robot)
  - [Dealing with the input](#Dealing-with-the-input)
  - [Experimentation to derive $Cl$ and $Cr$](#Experimentation-to-derive-$Cl$-and-$Cr$)
  - [Calcultating the covariance matrix of the motion model $Q_t$](#Calcultating-the-covariance-matrix-of-the-motion-model-$Q_t$)
  - [Deriving the observation model of the Thymio Robot](#Deriving-the-observation-model-of-the-Thymio-Robot)
  - [Calcultating the covariance matrix of the observation model $R_t$ ](#Calcultating-the-covariance-matrix-of-the-observation-model-$R_t$ )
  - [testImplementing the Extended Kalman Filter](#Implementing-the-Extended-Kalman-Filter)



- [Control law ](#Control-law)

- [Main](#Main)

- [The Code](#The-Code)

- [Sources](#Sources)
  



# **Demo videos**

The optimal path is displayed in white, while the real-time estimated position of the Thymio, obtained through filtering, is indicated by a black dotted line.

Full demo + Normal situation + Kidnapping + Camera obstructed + Local obstacles: https://youtu.be/rz_EOHwGMOw

Full demo: https://youtu.be/DTvLO2GhbcM
    
Normal situation: https://youtu.be/M8M3lAdUMPQ

Kidnapping: https://youtu.be/5M4zRgAHsiA

Camera obstructed: https://youtu.be/vcZKryRRfx0

Local obstacles: https://youtu.be/Q0z2UTtgOtY


# **Introduction**

#### **Project description and details:**
The project is structured around five distinct modules, each tailored to a specific aspect of the Thymio Robot's navigation and operational capabilities:
- Vision Module
- Global Navigation Module
- Local Navigation Module
- Filtering Module
- Robot control

The project uses mainly the camelCase naming convention for variables and functions (everything except the kalman filter). For the kalman filter, the snake case convention was used. This was because of the ditribution of the work among the team members.



#### **Modules and libraries required:**

The following modules and libraries are required to run the code:
- numpy
- matplotlib
- cv2
- math
- time
- pyvisgraph
- tdmclient

# **Environment**

For the chosen environment, we have selected a monochromatic blue background, complemented by 2D green obstacles and a yellow target area. The Thymio robot is identified by a distinctive red "mask" applied to it. The environment was constructed on a pool table.

<div align="center">
    <img src="img/environment.jpeg" alt="Environment on the pool table" width="400" style="margin-right: 10px;"/>
    <img src="img/thymio.jpeg" alt="Thymio with the mentioned mask" width="400"/>
</div>


In the development of our Mobile Robotics Project, we employed a forward-thinking strategy by incorporating ChatGPT-4's capabilities. Our choice was influenced by ChatGPT-4's sophisticated computational prowess and its adeptness in crafting intricate geometric figures. We primarily utilized the model to construct a variety of polygons, which functioned as fixed obstacles on our map. We observed that while the model could produce a broad spectrum of shapes, from simple triangles to elaborate polygons, it encountered difficulties in creating a PDF with non-overlapping shapes. Additionally, it was challenged in randomly generating polygons, often resulting in blank outputs or error messages. Consequently, we intervened by specifying the dimensions, forms, and hues of the polygons. As a result, our map was not entirely AI-generated but rather a collaborative effort, blending our specific polygon choices with GPT-4's assembly skills. This experience once again underscored the limitations of a Large Language Model (LLM) in tasks demanding significant creativity.

<div align="center">
    <img src="img/GPT_error.png" alt="GPT error message" width="450" style="margin-right: 10px;"/>
    <img src="img/GPT_env.png" alt="polygons generated by GPT4 " width="550"/>
</div>

# **Vision**

#### **Camera**

<div align="center">
    <img src="img/iPhone.jpg" alt="very bad image of the concept" width="400"/>
</div>

The camera used in the project was not the USB camera provided by the course. We started the development with the USB camera, but while experimenting with the camer choice we discovered that using a macbook, the camera of the iphone with the same icloud account is automatically recognized as a webcam wirelessly. This was great considering the camera had to be placed in the roof pointing down at the table. This allowed us to stick the phone to the roof and not have any cables hanging from the roof to the computer. From python, the camera is recognized as a normal webcam, so the code did not have to be changed.

#### **Environment mapping**

The computer vision module maps the environment to a 2d plane, to allow for correct functioning of the following modules. To do this, four small green markers are placed on the corners of the table. The camera detects the markers (the global obstacles are also green so the external 4 green detected objects are taken) and maps these point to certain point on the plane depending on the aspect ratio of the markers.

### **Global obstacle detection**

The global obstacle detection is done by detecting the green objects in the environment. These objects are filtered by size and are expanded since the global navigation algorithm used is the visibility graph.

### **Robot detection**

The robot is found by detecting the red mask on the robot. The mask is a composed of a white background and two red rectangles, one large in the back (center between the two wheels, important since this is the center of rotation) and one small in the front. Two rectangles are used to be able to discriminate the angle from 0 to 2pi instead of 0 to pi. This method gives us the robot's position and orientation.

### **Goal detection**
The goal is detected using a yellow detector. The centroid of the largest yellow object is taken as the goal position.



# **Global Navigation**

#### **Visibility graph**
For the global navigation, we used the visibility graph algorithm. To do this we used the library pyvisgraph. The library has all the functions needed including the finding of the shortest path, so no extra libraries (apart from the already used numpy and matplotlib) were needed. It is important to note that the library uses a different coordinate standard than openCV, so an extra function was needed to convert the coordinates. Another important addition was the pushing far of the point in the edge. When initially testing the system we encountered issues when a global obstacle was close to the edge even if expanded for the robot size. To resolve this we pushed the points of the global obstacles that are on the edge to a distance far away from the center. This ensured that the path planning algorithm would not consider them.



<div align="center">
    <img src="img/pointsInPath.png" alt="Plot with points in the shortest path" width="400" style="margin-right: 10px;"/>
</div>


# **Local Navigation**

#### **Local obstacle detection**

To detect the local obstacles we used the horizontal distance sensors in the front of the robot. Depending on in which side the obstacle is detected, the robot will turn in the opposite direction and avoid the obstacle using a squared path manouver. The moving of the robot was done using a direct speed input and timers, since it proved to be simple but effective. The values of the timers were found by trial and error. Even if considerably precise, at the end of the manouver and once the obstacle has been avoided, the global path planning algorithm is run again to ensure the continuation is smooth and the robot recovers in th ebest possible manner.  This method has some limitations though. The aproximated size of the obstacles has to be known a priori and sufficient place to do the manouver has to be available.

#### **Kidnaping**


To detect the kidnaping of the robot we use the vertical distance sensors on the robot. If the robot is lifted from the table a kidnaped flag is activated and while the robot is not positioned on the tale the robot turns off the motors and waits. When the robot is placed back in the table, a few seconds are waited until the robot is stable and no other perturbations are present (hands that can confuse the robot location for example). One this is the case, the global path planning algorithm is run again to find the new path to the goal. 

We used the vertical distance sensors in the ground reflected mode. After experimentation we remarked that this was the mode that showed the most difference when the robot was placed on the table or lifted from it, facilitating the determination of the threshold.

# **Filtering: Extended Kalman Filter**

#### **Why Kalman?**

During the course we have encountered different Bayes filter to estimate the state of a robot in a given environment. For example, we could have used a particle filter, but it is more computationally expensive and it is not necessary for this application. For our task we have choosen the Kalman filter, a recursive filter that estimates the state of a system from a series of noisy measurements. It is a very efficient algorithm that can be used to solve the localization problem of a robot, but has a crucial assumption about the distribution of the noise. In fact, it assumes that the noise is Gaussian distributed, which is not always the case in real life. However, in differential robots, like the Thymio, odometry is primarily based on the velocity sensors of the motors to calculate the robot's position and orientation ($x, y, \gamma$). The errors in odometry can arise from various sources:
- Sensor Measurement Errors: The sensors might not be perfectly accurate or might exhibit some drift over time.
- Mechanical Misalignments: Slight imperfections in the robot's construction or wear in components can cause discrepancies.
- Differential Motion Transmission Variances: Factors like friction, tire pressure, or differences in the movement surface can affect how motion is transmitted from the motors to the wheels.
- Slippage Error: The wheels may slip or lose traction, leading to inaccurate distance measurements.

Since each of these error sources contributes small, independent increments to the overall odometry error. According to the Central Limit Theorem, when a large number of small, independent effects are summed, their overall distribution tends to approach a Gaussian distribution. This implies that even if each individual source of error might not be Gaussian, their cumulative effect tends towards a normal distribution. 

Since the odometry noise can be assumed to be Gaussian distributed, the Kalman filter, that's easy to implement, is a good choice for our application.

Once again we have decied to collaborate with LLM. This time we wanted to generate an image suitable for this paragraph, but even if we were precise about the contest ChatGPT-4 was able to produce an image not appropriated for this paragraph, the image is not scientific or accurated but we decided to use it anyway, to highlight another limit of LLM.

<div align="center">
    <img src="img/GPT_Kalman.png" alt="very bad image of the concept" width="400"/>
</div>


#### **Deriving the state space model of the Thymio Robot**

Source: https://automaticaddison.com/how-to-derive-the-state-space-model-for-a-mobile-robot/ 

<div align="center">
    <img src="img/state_space_.png" alt="2D image of a differential drive robot" width="400"/>
</div>


First of all, we need to define the state and input variables. We will use the following state variables: $x$ and $y$ for the position, and $\gamma$ for the orientation, since we can control our Thyimio in speed, we can define our input variables $v$ forward velocity, and $\omega$ angular velocity . After some simple trigonometric analysis is easy to define the state vector $s$ is defined as follows: 
$$
s_{t} =
\begin{bmatrix}
x_{t} \\
y_{t} \\
\gamma_{t}
\end{bmatrix}
=
\begin{bmatrix}
x_{t-1} + v_{t-1}\cos\gamma_{t-1} \cdot \Delta t \\
y_{t-1} + v_{t-1}\sin\gamma_{t-1} \cdot \Delta t \\
\gamma_{t-1} + \omega_{t-1} \cdot \Delta t
\end{bmatrix}
=
\begin{bmatrix}
f_1 \\
f_2 \\
f_3
\end{bmatrix}
$$

Hence our input vector $u$ is defined as follows:

$$
u_{t} = 
\begin{bmatrix}
    v_{t} \\ 
     \omega_{t}
\end{bmatrix}
$$
We would now define our state space model as our system is linaer so in this form:  $s_{t}= A_{t-1}s_{t-1} + B_{t-1}u_{t-1}$, where $A_{t-1}$ is the state matrix and $B_{t-1}$ is the input matrix. But our original system is not linear. However, we can approximate our system by using the Jacobian matrix.
Our Thymio moves only when it receives instructions to rotate its wheels. As a result, in this scenario, the $A_{t-1}$ matrix is an identity matrix (linear relation). For the $B_{t-1}$ matrix we can use the Jacobian matrix, which is defined as follows:

$$
B_{t-1} = 
\begin{bmatrix}
\frac{\partial f_1}{\partial v_{t-1}} & \frac{\partial f_1}{\partial \omega_{t-1}} \\
\frac{\partial f_2}{\partial v_{t-1}} & \frac{\partial f_2}{\partial \omega_{t-1}} \\
\frac{\partial f_3}{\partial v_{t-1}} & \frac{\partial f_3}{\partial \omega_{t-1}}
\end{bmatrix}
=
\begin{bmatrix}
\cos(\gamma_{t-1} \cdot \Delta t) & 0 \\
\sin(\gamma_{t-1} \cdot \Delta t) & 0 \\
0 & \Delta t
\end{bmatrix}
$$
If we consider also the process noise $ w_{t} = [ w^1_{t}, w^2_{t}, w^3_{t} ] $
 , we can define our state space model as follows:
$$s_{t}= A_{t-1}s_{t-1} + B_{t-1}u_{t-1} +w_{t-1}$$
$$
\begin{bmatrix}
x_{t} \\
y_{t} \\
\gamma_{t}
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x_{t-1} \\
y_{t-1} \\
\gamma_{t-1}
\end{bmatrix}
+
\begin{bmatrix}
\cos(\gamma_{t-1} \cdot \Delta t) & 0 \\
\sin(\gamma_{t-1} \cdot \Delta t) & 0 \\
0 & \Delta t
\end{bmatrix}
\begin{bmatrix}
v_{t-1} \\
\omega_{t-1}
\end{bmatrix}
+
\begin{bmatrix}
w^1_{t-1} \\
w^2_{t-1} \\
w^3_{t-1}
\end{bmatrix}
$$


Note that we are assuming that the process noise is zero-mean Gaussian noise with covariance matrix $Q_k$, which is computed after some experimentation explained in the next section.



#### **Dealing with the inputs**

Source: https://www.youtube.com/watch?v=aE7RQNhwnPQ

If we notice, our state space model is controlled in $v$ forward velocity, and $\omega$ angular velocity but in reality we can only control the speed of the two motors $v_{right}$, and $v_{left}$ of our Thymio. So we need to find a way to convert the speed of the robot $v$ and $\omega$ into $v_{right}$, and $v_{left}$. We can do this by using the following equations:

$v_{right} = \frac{2v+\omega L}{2R}$ , $v_{left} = \frac{2v-\omega L}{2R}$

Where $L$ is the distance between the two wheels and $R$ is the radius of the wheels.

In order to control the Thymio we must convert the speed of the two motors from the Thymio unit system to $v_{right}$, and $v_{left}$  we can do this buy using the convertion factor $Cr$ and $Cl$ calculated through experimentation.




#### **Experimentation to derive $Cl$ and $Cr$**

In order to use the $Cl$ and $Cr$ values in the program, we need to derive them from the experimental data. This section contains the collected data and the results. First we have marked each wheel of the Thymio robot then we have run the robot at a fixed speed $spd = 200$  $Thymio units$ and measured the number of revolution of each wheel in a fixed time interval $t$. We have repeated this experiment 10 times and calculated our average values for $Cl$ and $Cr$.
Since we have noticed that $Cl$ and $Cr$ are not proportional with the Thymio motors speed, we have decided to use the speed used to derive this coefficent as the constant speed input in our controller.


<div align="center">
    <img src="img/Thymio_wheel.jpeg" alt="Description of Thymio wheel" width="500" style="margin-right: 10px;"/>
    <img src="img/Cl_Cr.png" alt="Description of second image" width="300"/>
</div>



$$Cl = 69.33821285 \frac{s}{rad}Thymio units,Cr = 70.74580573 \frac{s}{rad}Thymio units$$



#### **Calcultating the covariance matrix of the motion model $Q_t$**


$$
Q_t = 
\begin{bmatrix}
Var(x_t) & Cov(x_t, y_t) & Cov(x_t, \gamma_t) \\
Cov(y_t, x_t) & Var(y_t) & Cov(y_t, \gamma_t) \\
Cov(\gamma_t, x_t) & Cov(\gamma_t, y_t) & Var(\gamma_t)
\end{bmatrix}
$$

To implement a Kalman filter, we first need to know the variance of the motion model. 
While my odometry system provides only the velocity of the two motors, we can use the geometry of the Thymio to calculate the position and orientation of the robot, hence: $$x=f_1(v_r,v_l) = \frac{R}{2} (v_r + v_l) \cos(\gamma) \Delta t$$ $$y=f_2(v_r,v_l) = \frac{R}{2} (v_r + v_l) \sin(\gamma) \Delta t$$ $$\gamma=f_3(v_r,v_l) =\frac{R}{L} (v_r-v_l) \Delta t$$ where $v_r$ and $v_l$ are the velocity of the two motors, $R$ is the radius of the wheel and $L$ is the distance between the two wheels . Now we have to take in account the error propagation, so we can calculate $$Var(x) = (\frac{\partial f_1}{\partial v_r})^2 Var(v_r) + (\frac{\partial f_1}{\partial v_l})^2 Var(v_l) +2(\frac{\partial f_3}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l) $$ $$Var(y) = (\frac{\partial f_2}{\partial v_r})^2 Var(v_r) + (\frac{\partial f_2}{\partial v_l})^2 Var(v_l) +2(\frac{\partial f_3}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l) $$ $$Var(\gamma) = (\frac{\partial f_3}{\partial v_r})^2 Var(v_r) + (\frac{\partial f_3}{\partial v_l})^2 Var(v_l) +2 (\frac{\partial f_3}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l)  $$ 

We can consider $v_r$ and $v_l$ uncorrelated so after deriving the partial derivatives we can calculate the diagonal terms of $Q_t$ as follows:

$$Var(x) =  (\frac{R}{2} \cos(\gamma) \Delta t)^2 (Var(v_r)+Var(v_l))$$ 
$$Var(y) = (\frac{R}{2} \sin(\gamma) \Delta t)^2 (Var(v_r)+Var(v_l))$$ 
$$Var(\gamma) =  (\frac{R}{L}\Delta t)^2 (Var(v_r)+Var(v_l))$$

Next we need to calculate the off-diagonal terms of $Q_t$, we can do it again by taking in account the error propagation:
$$ Cov(x,y) = Cov(y,x)  = (\frac{\partial f_1}{\partial v_r}) (\frac{\partial f_2}{\partial v_r}) Var(v_r) + (\frac{\partial f_1}{\partial v_l}) (\frac{\partial f_2}{\partial v_l}) Var(v_l) + (\frac{\partial f_1}{\partial v_r}) (\frac{\partial f_2}{\partial v_l}) Cov(v_r,v_l) + (\frac{\partial f_1}{\partial v_l}) (\frac{\partial f_2}{\partial v_r}) Cov(v_l,v_r)$$
$$ Cov(x,\gamma) = Cov(\gamma,x) = (\frac{\partial f_1}{\partial v_r}) (\frac{\partial f_3}{\partial v_r}) Var(v_r) + (\frac{\partial f_1}{\partial v_l}) (\frac{\partial f_3}{\partial v_l}) Var(v_l) + (\frac{\partial f_1}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l) + (\frac{\partial f_1}{\partial v_l}) (\frac{\partial f_3}{\partial v_r}) Cov(v_l,v_r)$$
$$ Cov(y,\gamma) = Cov(\gamma,y) = (\frac{\partial f_2}{\partial v_r}) (\frac{\partial f_3}{\partial v_r}) Var(v_r) + (\frac{\partial f_2}{\partial v_l}) (\frac{\partial f_3}{\partial v_l}) Var(v_l) + (\frac{\partial f_2}{\partial v_r}) (\frac{\partial f_3}{\partial v_l}) Cov(v_r,v_l) + (\frac{\partial f_2}{\partial v_l}) (\frac{\partial f_3}{\partial v_r}) Cov(v_l,v_r)$$

After deriving the partial derivatives we obatain:

$$ Cov(x,y) = \frac{(R  \Delta t)}{4}^2(\cos(\gamma)\sin(\gamma))(Var(v_r)+Var(v_l)) $$
$$ Cov(x,\gamma) = \frac{(R \Delta t)^2}{2L} \cos(\gamma) (Var(v_r)-Var(v_l) )$$
$$ Cov(y,\gamma) = \frac{(R \Delta t)^2}{2L} \sin(\gamma) (Var(v_r)-Var(v_l)) $$

Now that we have our covariance matrix $Q_k$ formulation, we can calulate the parameters $Var(v_r)$ and $Var(v_l)$ by experimentation. By simply collecting data from our Thymio proceding at a constant speed of $200$ $Thymiounits$, we have obtained the variance of the two motors, in $(rad/s)^2$ 


<div align="center">
    <img src="img/motor_speed.png" alt="Description of the image" width="400"/>
</div>

$$Var(v_r) = 0.013997244458988999 (rad/s)^2$$
$$Var(v_l) = 0.007137898281767495 (rad/s)^2$$

After some simulation for different values of $\gamma$ from $0°$ to $180°$ we have found that the matrix $Q_t$ is diagonal.
<div align="center">
    <img src="img/Q.png" alt="Description of the image" width="400"/>
</div>

To ensure that the Kalman filter minimizes the variance of the error between the actual state and the predicted state, it is necessary to have a $Q$ matrix that is not time-varying ($Q_t = Q$). To achieve this, it is essential to find a $Q$ matrix that takes scalar inputs. Through simulation, we have selected a $Q$ matrix with values that maximize the variance values, thereby obtaining a conservative estimate of the matrix.

$$
Q =
\begin{bmatrix}
0.20000 & 0.00000 & 0.00000 \\
0.00000 & 0.20000 & 0.00000 \\
0.00000 & 0.00000 & 0.00070
\end{bmatrix}

$$

Another crucial aspect to consider is that in our model the input contribution ($ B_{t-1}u_{t-1}$) represents a deterministic quantity. As a result, our current $Q$ matrix accounts solely for the state error, excluding any input error. A common practical approach in control theory is to incorporate the input error into an experimental variable, $\alpha$, which requires experimental tuning. Consequently, our actual $Q$ matrix becomes $Q$ =$\alpha  Q$, effectively encompassing the error associated with the input as well. After some tuning we have found that a good matrix $Q$ for our application is:

$$
Q = 

\begin{bmatrix}
2 & 0 & 0. \\
0& 2 & 0 \\
0 & 0 & 0.07
\end{bmatrix}
$$


#### **Deriving the observation model of the Thymio Robot**

Source: https://automaticaddison.com/how-to-derive-the-observation-model-for-a-mobile-robot/

Now that we have a linear state space model of our Thyimio that includes also the process noise, we need an observation model in order to be able to implement the Kalman filter. 

An observation model describes how expected sensor outputs $y_t$ are related to the state vector $s_t$, considering also a vector of observation noise $ \nu_{t} = [ \nu^1_{t}, \nu^2_{t}, \nu^3_{t} ] ^T$
with zero mean and covariance matrix $R$ (which is computed after some experimentation explained in the next section). 
The observation model is defined as follows: 
$$y_t = H_t s_t + \nu_t$$ 
Since we are able to detect the position and the orientation of the Thyimio in the environment using camera, our matrix $H$ is the identity, hence we can define our observation model as follows:

$$
y_{t} =
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}

\begin{bmatrix}
x_{t} \\
y_{t} \\
\gamma_{t}
\end{bmatrix}
+
\begin{bmatrix}
\nu^1_{t}\\
\nu^2_{t}\\
\nu^3_{t}
\end{bmatrix}
$$

#### **Calcultating the covariance matrix of the observation model $R_t$**

The matrix $R_t$ is obtain by experimentation, and contain the variance of the observation noise, in our case the variance of the position and orientation of the Thymio in the environment, we can denote the observation vector as follows:

$$
z_{t} =
\begin{bmatrix}
x^o_{t} \\
y^o_{t} \\
\gamma^o_{t}
\end{bmatrix} 
$$

We can calculate the variance of the observation and it will be:

$$
R_t = 
\begin{bmatrix}
Var(x^o) & Cov(x^o, y^o) & Cov(x^o, \gamma^o) \\
Cov(y^o, x^o) & Var(y^o) & Cov(y^o, \gamma^o) \\
Cov(\gamma^o, x^o) & Cov(\gamma^o, y^o) & Var(\gamma^o)
\end{bmatrix}
$$
Note that those term are different from the ones of the motion model, because the observation noise is releted with our camera and our image vision approach, not with the Thymio itself. This matrix changes in relation with the visibility of our Thymio, for simplicity we can assume two different scenarios, when the Thymio is visible and when it is not, so we can define just 2 matrix $R$ and $R_{nc}$.If we can not detect it the incertence grows to infinity, so we can define a matrix $R_{nc}$, in this case we set the variance of the observation noise to infinity:
$$
R_{nc} = 
\begin{bmatrix}
\infty &0 & 0 \\
0 & \infty & 0 \\
0 & 0 & \infty 
\end{bmatrix}
$$

Now I can compute $R_t = R$ through experimentation, first we need to create an A4 paper as a calibration grid, ensuring the most accurate positioning of the Thymio robot before detecting its location and orientation. The A4 paper was easly generated using ChatGPT-4. The LLM after getting as input the shapes and the dimension needed for the grid was able to generate the following file:

<div align="center">
    <img src="img/square_design.png" alt="Description of the image" width="400"/>
</div>

Then we have computed the required variance mesuring the delta between the measure of the Thymio provided by the camera and the real position of the Thymio. We have repeated this 10 times and calculated our average values for $Var(x^o)$, $Var(y^o)$ and $Var(\gamma^o)$. To be consistent with the units of the motion model, we have converted the camera measure from pixels to millimiters.

<div align="center">
    <img src="img/R_exp.jpg" alt="Description of the image" width="400"/>
</div>




$$Var(x^o) = 0.7 (mm)^2$$
$$Var(y^o) = 0.7 (mm)^2$$
$$Var(\gamma^o) = 0.0014 (rad)^2$$

Hence our $R$ matrix is:
$$
R = 
\begin{bmatrix}
0.7 &0 & 0 \\
0 & 0.7 & 0 \\
0 & 0 & 0.0014
\end{bmatrix}
$$


#### **Implementing the Extended Kalman Filter**

Source: https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/

Since our model is not linear (the $B$ matrix has a trigonometrical relation with the state varaiable $\gamma$), we have to implement an Extended Kalman Filter

Due to the previous consideration we will have both the $R$ and the $R_{nc}$ to taking in to account the possibility of the Thymio being visible or not. We will use the $R_{nc}$ when the Thymio is not visible and $R$ when it is visible.

For the implementation we would refer to the measurements of the position and orientation of the Thymio in the environment obtained using the camera as 

$$
z_{t} =
\begin{bmatrix}
x_{t} \\
y_{t} \\
\gamma_{t}
\end{bmatrix} 
$$


# **Robot control**

Initially, a simple P controller was employed for the control law, aiming to minimize the perpendicular distance from the robot to the ideal line. However, this controller led to instability issues. Consequently, we made the decision to switch to a proportional-integral (PI) controller for the angle and a proportional (P) controller for the distance to the optimal path.

This controller functions based on the angle error between the desired angle and the current angle of the Thymio robot, along with the perpendicular distance from the Thymio to the line it should be following. Two main modes govern the controller's operation: alignment mode and forward mode.

Alignment Mode: In this mode, the controller exclusively addresses the angle error. It is utilized when the robot reaches each point in the path, preparing for the next segment.
Forward Mode: In this mode, the controller considers both the angle error and the perpendicular distance to the line. It becomes active when the robot is in motion between two points in the path.

<div align="center">
    <img src="img/control.png" alt="Control error diagram" width="300"/>
</div>

# **Main**

The main function is the core of the project. It is responsible for the initialization of the different modules and the communication between them.

The function follows the concept of operations presented in the figure below:

<div align="center">
    <img src="img/conOps.png" alt="Cocept of operations" width="800"/>
</div>

# **The Code**

In [1]:
import numpy as np
import time
import cv2

from src.computerVision import correctPerspectiveStream, findCorners, getPerspectiveMatrix, findThymio, findGlobalObstacles, findGoal
from src.kalman import estimatePosition, inverseSpeedConversion, r11, r22, r33, Cl, Cr, R, L
from src.pathPlanning import buildGraph
from src.robotControl import robotController, checkForObstacles

XYMIRROR = False
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080
POSITION_THRESHOLD = 50
KIDNAPPING_THRESHOLD = 100
KIDNAPPING_TIME = 5


In [ ]:
# Note: This function was provided by the course Basics Of Mobile Robotics by Prof. Francesco Mondada
# Source: https://moodle.epfl.ch/course/view.php?id=15293

import tdmclient.notebook
await tdmclient.notebook.start()



@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):

    global motor_left_target, motor_right_target

    l_speed = int(l_speed)
    r_speed = int(r_speed)
    
    motor_left_target = l_speed
    motor_right_target = r_speed

In [ ]:
# Note: This function was provided by the course Basics Of Mobile Robotics by Prof. Francesco Mondada
# Source: https://moodle.epfl.ch/course/view.php?id=15293

@tdmclient.notebook.sync_var 
def horiz_sensor():
    global prox_horizontal
    return prox_horizontal

In [ ]:
# Note: This function was provided by the course Basics Of Mobile Robotics by Prof. Francesco Mondada
# Source: https://moodle.epfl.ch/course/view.php?id=15293

@tdmclient.notebook.sync_var 
def getVerticalDistance():
    global prox_ground_reflected
    return prox_ground_reflected


In [ ]:
# Note: This function was provided by the course Basics Of Mobile Robotics by Prof. Francesco Mondada
# Source: https://moodle.epfl.ch/course/view.php?id=15293

%%run_python
nf_leds_prox_h(0,0,0,0,0,0,0,0) 
nf_leds_rc(0)
nf_leds_temperature(0, 0)
nf_leds_bottom_right(0,0,0)
nf_leds_bottom_left(0,0,0)
nf_leds_top(0,0,0)
nf_leds_prox_v(0,0)


In [ ]:
#Note: these functions are defined here and not in their respective files because they call the functions that interact with the robot defined above

def checkForKidnap():
    ver=getVerticalDistance()
    if ver[0] < KIDNAPPING_THRESHOLD or ver[1] < KIDNAPPING_THRESHOLD:
        print("Kidnapped!")
        motors(0,0)
        return True
    else:
        return False

def avoidObstacle(leftOrRight, initTime, currentTime):
    motors(0,0)
    
    tTurn = 1
    tStraight = 2
    vStraight = 200

    if leftOrRight == 'right':
        l=-200
        r=200
    elif leftOrRight == 'left':
        l=200
        r=-200

    motors(l_speed=l, r_speed=r)
    if currentTime - initTime < tTurn:
        return True

    motors(l_speed=vStraight, r_speed=vStraight)
    if currentTime - initTime < tTurn + tStraight:
        return True
    

    motors(l_speed=-l, r_speed=-r)
    if currentTime - initTime < 2*tTurn + tStraight:
        return True

    motors(l_speed=vStraight, r_speed=vStraight)
    if currentTime - initTime < 2*tTurn + 2.5*tStraight:
        return True

    motors(l_speed=-l, r_speed=-r)
    if currentTime - initTime < 3*tTurn + 2.5*tStraight:
        return True

    motors(l_speed=vStraight, r_speed=vStraight)
    if currentTime - initTime < 3*tTurn + 3.5*tStraight:
        return True

    motors(l_speed=l, r_speed=r)
    if currentTime - initTime < 4*tTurn + 3.5*tStraight:
        return True

    motors(l_speed=0, r_speed=0)

    return False


In [ ]:
cap = cv2.VideoCapture(1)
previousTime = time.time()

#state control variables
correctedCam = False
environmentSetup = False
redoPath = False
avoidingObstacle = False
wasKidnapped = False
aligned = False

#initializing variables
path = []
goal = np.array([0,0])
lSpeed = 0
rSpeed = 0
P_k = np.array([[r11,0,0],[0,r22,0],[0,0,r33]])
postionHistory = []


while True:
    # Read a frame from the video stream
    key = cv2.waitKey(1)
    ret, frame = cap.read()


    if not ret:
        print("Error: failed to capture image")
        break

    if not correctedCam:
        # Map the image to the corner
        position, angle, _=findThymio(frame)
        estimateState = np.array([position[0],IMAGE_HEIGHT-position[1],angle])
        cv2.imshow('Thymio Camera', frame)
        if position[0] == -1:
            print("thymio where are you?")
            continue
        try:
            centroids = findCorners(frame)
            perspectiveMatrix = getPerspectiveMatrix(centroids)
            correctedCam = True
        except:
            print("Fail")
            continue
            
    
    
    # Correct the perspective of the image
    if correctedCam:
        frame = correctPerspectiveStream(frame,perspectiveMatrix)
        frameToPlot = frame



    current_time = time.time()
    dt = current_time - previousTime
    if dt < 0.15:
        time.sleep(0.15-dt)

    previousTime = current_time


    previousControlVector = inverseSpeedConversion(rSpeed,lSpeed,R,L,Cr,Cl)

    position, angle, estimateState, P_k = estimatePosition(frame,previousControlVector,dt,P_k,estimateState)

    postionHistory.append(position)



    #plot the points in position history in black
    for i in range(len(postionHistory)-1):
        cv2.circle(frameToPlot, (int(postionHistory[i][0]), int(IMAGE_HEIGHT-postionHistory[i][1])), 5, (0, 0, 0), -1)

    #plot the goal
    cv2.circle(frameToPlot, (int(goal[0]), int(goal[1])), 5, (0, 0, 255), -1)

    #plot the path lines
    for i in range(len(path)-1):
        cv2.line(frameToPlot,(int(path[i][0]),int(IMAGE_HEIGHT-path[i][1])),(int(path[i+1][0]),int(IMAGE_HEIGHT-path[i+1][1])),(255,255,255),2)
        cv2.circle(frameToPlot, (int(path[i][0]), int(IMAGE_HEIGHT-path[i][1])), 5, (255, 255, 255), -1)
        
    cv2.imshow('Thymio Camera', frameToPlot)
    
    
    if checkForKidnap():
        #cambiare le condizioni
        print("kidnapped")
        lSpeed = 0
        rSpeed = 0
        wasKidnapped = True
        continue


    if wasKidnapped:
        time.sleep(KIDNAPPING_TIME)
        wasKidnapped = False
        redoPath = True
        continue


    if not avoidingObstacle:
        leftOrRight = checkForObstacles(horiz_sensor())


    if leftOrRight != None or avoidingObstacle:
        if not avoidingObstacle:
            initTime = time.time()
        avoidingObstacle = avoidObstacle(leftOrRight, initTime, time.time())
        redoPath = True
        continue
    

    if (key == ord(' ') and environmentSetup == False) or redoPath:
        # Capture an image
        imagePath = 'capturedImage.jpg'
        cv2.imwrite(imagePath, frame)

        capturedImage = cv2.imread(imagePath)
        position, angle, estimateState, P_k = estimatePosition(capturedImage,previousControlVector,dt,P_k,estimateState)

        greenObjects = findGlobalObstacles(capturedImage)

        goal = findGoal(capturedImage)
        
        print(goal)

        cv2.imshow('Thymio Camera', capturedImage)
        

        path=buildGraph(greenObjects,position,goal)
        pointCount = 0
        print(path)

        if redoPath:
            redoPath = False
            continue
        environmentSetup = True
        
        #cv2.waitKey(0)  # Wait until any key is pressed to close the image window



    if environmentSetup:

        if np.linalg.norm(position - path[pointCount]) < POSITION_THRESHOLD:
            aligned = False
            pointCount += 1
            motors(0,0)

            if pointCount == len(path):
                motors(0,0)
                print("Goal reached")
                break

            continue
        
        if not aligned:
            lSpeed,rSpeed,distanceError,angleError = robotController(path[pointCount-1],path[pointCount],position,angle,dt,alignMode=True)
            motors(lSpeed,rSpeed)
            if np.abs(angleError) < 0.1:
                aligned = True
                motors(0,0)
            continue


        lSpeed,rSpeed,distanceError,angleError = robotController(path[pointCount-1],path[pointCount],position,angle,dt,alignMode=False)
        motors(lSpeed,rSpeed)

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
motors(0,0)

# **Sources**


Chat-GPT4: https://chat.openai.com/

Copilot: https://copilot.github.com/

Automatic Addison: https://automaticaddison.com

mouhknowsbest (Youtube Channel): https://www.youtube.com/watch?v=aE7RQNhwnPQ

Course "Basics of Mobile Robotics" by Prof. Francesco Mondada (MICRO-452): https://moodle.epfl.ch/course/view.php?id=15293



